In [1]:
# Disable tensorflow warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# 0 = all messages are logged (default behavior)
# 1 = INFO messages are not printed
# 2 = INFO and WARNING messages are not printed
# 3 = INFO, WARNING, and ERROR messages are not printed

In [2]:
from model import MukkeBudeLSTM
from mapping import MusicMapping
import utils
import music21 as m21
import tensorflow as tf
import numpy as np
from pathlib import Path

# Check if GPU is found
print(tf.config.list_physical_devices('GPU'))

2023-04-04 11:06:48.559433: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


[]


In [3]:
# Create mappings
mapping = MusicMapping.create()
mapping.save("mapping.txt")

In [4]:
paths = m21.corpus.getComposer('bach')
paths = paths[:100]
print(f"Found {len(paths)} songs in corpus.")

encoded_songs = utils.load_dataset_lstm(paths, 64, mapping)

print(f"Songs encoded: {len(paths)}")

Found 100 songs in corpus.
Removed 5 bad songs
Remaining songs: 95
Songs encoded: 100


In [5]:
# Train model
model = MukkeBudeLSTM(mapping)
print(model)
model.train(encoded_songs)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, 298)]       0         
                                                                 
 lstm (LSTM)                 (None, 256)               568320    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense (Dense)               (None, 298)               76586     
                                                                 
Total params: 644,906
Trainable params: 644,906
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
1887/1887 [==============================] - 181s 95ms/step - loss: 1.2520 - accuracy: 0.7487
Epoch 2/50
1887/1887 [==============================] - 158s 83ms/step - loss: 1.0

In [6]:
model.save("lstm_bach")

INFO:tensorflow:Assets written to: lstm_bach.md5/assets


INFO:tensorflow:Assets written to: lstm_bach.md5/assets


In [13]:
# Create song
generated_song = model.generate("n72 _ _ _ _ _ n72 _ _ _ _ _ n72 _ n71 _", max_length=500)    
new_song = utils.decode_songs_old(generated_song)

1/1 [==============================] - 0s 23ms/step


In [14]:
path = Path("generated_song_bach_lstm.mid")
utils.write_midi(new_song, path)

In [16]:
# Open song in MuseScore (Linux if installed)
!musescore4portable {path.absolute()}

/lib/x86_64-linux-gnu/libjack.so.0
/lib/x86_64-linux-gnu/libnss3.so
QEventLoop: Cannot be used without QApplication
QEventLoop: Cannot be used without QApplication
13:36:48.520 | INFO  | main_thread | GlobalModule | onPreInit: log path: /home/deglasfl/.local/share/MuseScore/MuseScore4/logs/MuseScore_230404_133648.log
13:36:48.520 | INFO  | main_thread | GlobalModule | onPreInit: === Started MuseScore 4.0.1, build number 230121751 ===
13:36:48.528 | ERROR | main_thread | LanguagesService | setCurrentLanguage: Error loading translator "/tmp/.mount_musescOJ824l/share/mscore4portable-4.0//locale/qt_de.qm" 
13:36:48.678 | INFO  | main_thread | DiagnosticsModule | onInit: success start crash handler
13:36:48.681 | WARN  | main_thread | Qt         | QIODevice::read (QFile, "/home/deglasfl/.local/share/MuseScore/MuseScore4/shortcuts.xml"): device not open
13:36:48.681 | WARN  | main_thread | Qt         | QIODevice::read (QFile, "/home/deglasfl/.local/share/MuseScore/MuseScore4/midi_mappings.xm

# Train with new Dataformat from chinese dude

In [17]:
# Load songs
paths = m21.corpus.getComposer('bach')
paths = paths[:100]
print(f"Found {len(paths)} songs in corpus.")

encoded_songs = []
for path in paths:
    song = utils.read_single_from_corpus(path)
    encoded_song = utils.to_polyphonic_encoding(song, mapping)
    encoded_songs.extend(encoded_song)

Found 100 songs in corpus.


/home/deglasfl/repos/mukkeBude/mukkeBude/utils.py:298: StreamIteratorInefficientWarning: highestTime is not defined on StreamIterators. Call .stream() first for efficiency
  song.flat.getElementsByClass("Note").highestTime,
/home/deglasfl/repos/mukkeBude/mukkeBude/utils.py:299: StreamIteratorInefficientWarning: highestTime is not defined on StreamIterators. Call .stream() first for efficiency
  song.flat.getElementsByClass("Chord").highestTime,


In [18]:
# Train model
model = MukkeBudeLSTM(mapping)
print(model)
model.train(encoded_songs)

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, None, 298)]       0         
                                                                 
 lstm_1 (LSTM)               (None, 256)               568320    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 298)               76586     
                                                                 
Total params: 644,906
Trainable params: 644,906
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
1182/1182 [==============================] - 110s 92ms/step - loss: 1.8928 - accuracy: 0.4703
Epoch 2/50
1182/1182 [==============================] - 106s 90ms/step - loss: 1

In [19]:
# Save model
model.save("lstm_bach_polyphonie")

INFO:tensorflow:Assets written to: lstm_bach_polyphonie.md5/assets


INFO:tensorflow:Assets written to: lstm_bach_polyphonie.md5/assets


In [20]:
# Create song
generated_song = model.generate("xxbos n67 d4 n62 d4 n58 d4 n43 d4 xxsep d4 n67 d4 n62 d4 n58 d4 n55 d4 xxsep d4 n69 d4 n62 d4 n57 d4 n54 d4 xxsep")    

new_song_ints = mapping.numericalize(generated_song)
new_song = utils.from_polyphonic_encoding(np.array(new_song_ints), mapping, bpm=100)

path = Path("generated_song_bach_lstm.mid")
utils.write_midi(new_song, path)


1/1 [==============================] - 0s 23ms/step


IndexError: index 132 is out of bounds for axis 2 with size 128